In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install langchain==0.1.10
!pip install -U sentence-transformers
!pip install qdrant-client
!pip install -q datasets
!pip install streamlit

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294

In [2]:
# Load libraries
from sentence_transformers import SentenceTransformer
import numpy as np
from qdrant_client import models, QdrantClient
import pandas as pd
from datasets import load_dataset


## 1.- Libraries Sample Usage

### Sentence Transformet - Usage

In [ ]:
# Choose an embedding model
sbert_model = SentenceTransformer("bert-base-nli-mean-tokens")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Select the sentences to tranform into a vector
sentence_embeddings = sbert_model.encode(["I like pizza", "The pizza is delicious"])

In [ ]:
# final dimension of the embedding
sentence_embeddings[0].shape

(768,)

In [ ]:
# Create a cosine function to compare similarity
def cosine(u, v):
  return np.dot(u, v)/ (np.linalg.norm(u) * np.linalg.norm(v) )

In [ ]:
# Compare similarity across the sentences
cosine(sentence_embeddings[0], sentence_embeddings[1])

0.88690925

### Qdrant Client - Usage

Usage on how to create a Vector Database and extract a value

In [ ]:
# Create a qdrant client and select storage
qdrant = QdrantClient(":memory:")

# Choose an embedding model
encoder = SentenceTransformer("bert-base-nli-mean-tokens")

# Create a collection (database)
qdrant.recreate_collection(
    collection_name = "Test",
    vectors_config = models.VectorParams(
        size = encoder.get_sentence_embedding_dimension(), # Type of embedding to use
        distance = models.Distance.COSINE # Type of search function
    )
)

# Add values to the collection
qdrant.upsert(
    collection_name = "Test",
    wait=True,
    points = [
        models.PointStruct(
            id = 1,
            vector = encoder.encode("Cats").tolist(), # Key value to embedd and search
            payload = {
                'text': "Everyone in this world likes pizza" # Restult to retrive
            }
        ),
        models.PointStruct(
            id = 2,
            vector = encoder.encode("Pizza").tolist(),
            payload = {
                'text': "Everyone in this world likes Cats"
            }
        )
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# Search for a value in the datavase
search = qdrant.search(
    collection_name = "n",
    query_vector = encoder.encode("Pasta").tolist(), # Query used to search on the database
    limit = 1
)

In [ ]:
# Result of search
search

[ScoredPoint(id=2, version=0, score=0.6621523721100135, payload={'text': 'Everyone in this world likes Gorillas'}, vector=None, shard_key=None)]

## 2. Implementation (BERT Embeddings)

### 2.1 Load Data & Functions

Load Data

In [ ]:
# Load a dataset created for this usage
data = load_dataset("ManuelAlv/academic_conuseling")
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:80: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


Generating dataset split:   0%|          | 0/25 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13 [00:00<?, ? examples/s]

DatasetDict({
    dataset: Dataset({
        features: ['question', 'answer'],
        num_rows: 25
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 13
    })
})

In [ ]:
# Dataset used for the chatbot
bert_dataset = data['dataset'].to_pandas()
bert_dataset

,question,answer
0,How can I deal with the pressure of maintainin...,Maintaining a high GPA requires balancing your...
1,"I'm feeling overwhelmed, burnout by my coursew...",Feeling overwhelmed is a sign you need to take...
2,How can I manage my time more effectively?,Effective time management involves planning yo...
3,Support on mental health,Universities typically offer a range of mental...
4,How can I improve my concentration while study...,Enhancing concentration starts with minimizing...
5,What strategies can help me cope with academic...,Coping with academic failure involves reflecti...
6,How can I handle social stress and anxiety,Handling social anxiety involves practicing re...
7,What can I do to prevent procrastination?,Preventing procrastination requires understand...
8,How can I maintain a work and life balance,Maintaining a healthy work-life balance involv...
9,How can I deal with homesickness at university?,Dealing with homesickness involves creating a ...


In [ ]:
# Dataset used to test the chatbot
test_dataset = data['test'].to_pandas()
test_dataset.columns = ["test_question", "original_question"]
test_dataset

,test_question,original_question
0,I'm stress of getting high school marks?,How can I deal with the pressure of maintainin...
1,I'm starting to feel burnt out from all my cla...,"I'm feeling overwhelmed, burnout by my coursew..."
2,Can you suggest ways to plan my study time bet...,How can I manage my time more effectively?
3,Where can I find help for mental health issues...,Support on mental health
4,What can I do to focus better during my study ...,How can I improve my concentration while study...
5,How do I bounce back from failing an exam?,What strategies can help me cope with academic...
6,What advice do you have for dealing with fear ...,How can I handle social stress and anxiety
7,How can I stop putting off my assignments unti...,What can I do to prevent procrastination?
8,how can I balance fun and school work,How can I maintain a work and life balance
9,How do I handle feeling isolated while studyin...,How can I deal with homesickness at university?


Create Functions

In [ ]:
# function to add values
def add_value(collection, key, value, id):
  encoder = SentenceTransformer(collection)

  qdrant.upsert(
      collection_name = collection,
      wait=True,
      points = [
          models.PointStruct(
              id = id,
              vector = encoder.encode(key).tolist(),
              payload = {
                  'text': value,
                  'question': key
              }
          )
      ]
  )

In [ ]:
# Function to search for a value
def search(collection, query):
  search = qdrant.search(
      collection_name = collection,
      query_vector = encoder.encode(query).tolist(),
      limit = 1
  )
  return search

### 2.2 Test Different Embedding Models

Create a Vector Database for each Embedding Model

In [ ]:
# Different BERT Embedding models available to use
MODEL_1 = "bert-base-nli-mean-tokens"
MODEL_2 = "all-distilroberta-v1"
MODEL_3 = "sentence-t5-base"
MODEL_4 = "all-MiniLM-L6-v2"
TEST_MODELS = [MODEL_1, MODEL_2, MODEL_3, MODEL_4]

# Create a local Vector Database
qdrant = QdrantClient(":memory:")

# Loop across the different models to test
for model in TEST_MODELS:
  print(f"**** Loading Model: {model} *******")
  encoder = SentenceTransformer(model)

  # Create a collection for the model an using its embeddings
  qdrant.recreate_collection(
      collection_name = model,
      vectors_config = models.VectorParams(
          size = encoder.get_sentence_embedding_dimension(),
          distance = models.Distance.COSINE
      )
  )

  # Add the data to each model
  for index, row in bert_dataset.iterrows():
    key = row['question']
    value = row['answer']
    id = index + 1

    add_value(model, key, value, id)

**** Loading Model: bert-base-nli-mean-tokens *******
**** Loading Model: all-distilroberta-v1 *******
**** Loading Model: sentence-t5-base *******


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

**** Loading Model: all-MiniLM-L6-v2 *******


Test on the sample data

In [ ]:
# Create a new dataset to store the results
result_dataset = test_dataset.copy()

# Loop across each model
for model in TEST_MODELS:
  print(f"**** Testing Model: {model} *******")
  encoder = SentenceTransformer(model)
  result_dataset[model] = "hi"

  # Test the retrival accruacy of each model on test data
  for index, row in test_dataset.iterrows():
    query = row["test_question"]
    result = search(model, query)
    result = result[0].payload['question']
    result_dataset[model].iloc[index] = result


**** Testing Model: bert-base-nli-mean-tokens *******
**** Testing Model: all-distilroberta-v1 *******
**** Testing Model: sentence-t5-base *******
**** Testing Model: all-MiniLM-L6-v2 *******


In [ ]:
# Visualize the retrived questions of each model
result_dataset

,test_question,original_question,bert-base-nli-mean-tokens,all-distilroberta-v1,sentence-t5-base,all-MiniLM-L6-v2
0,I'm stress of getting high school marks?,How can I deal with the pressure of maintainin...,How can I deal with the pressure of maintainin...,How can I deal with the pressure of maintainin...,How can I deal with the pressure of maintainin...,How can I deal with the pressure of maintainin...
1,I'm starting to feel burnt out from all my cla...,"I'm feeling overwhelmed, burnout by my coursew...","I'm feeling overwhelmed, burnout by my coursew...","I'm feeling overwhelmed, burnout by my coursew...","I'm feeling overwhelmed, burnout by my coursew...","I'm feeling overwhelmed, burnout by my coursew..."
2,Can you suggest ways to plan my study time bet...,How can I manage my time more effectively?,How can I improve my concentration while study...,How can I manage my time more effectively?,How can I manage my time more effectively?,How can I manage my time more effectively?
3,Where can I find help for mental health issues...,Support on mental health,How can I improve my sleep quality as a student?,Support on mental health,Support on mental health,Support on mental health
4,What can I do to focus better during my study ...,How can I improve my concentration while study...,How can I improve my concentration while study...,How can I improve my concentration while study...,How can I improve my concentration while study...,How can I improve my concentration while study...
5,How do I bounce back from failing an exam?,What strategies can help me cope with academic...,What strategies can help me cope with academic...,What strategies can help me cope with academic...,What strategies can help me cope with academic...,What strategies can help me cope with academic...
6,What advice do you have for dealing with fear ...,How can I handle social stress and anxiety,How can I handle social stress and anxiety,How can I handle social stress and anxiety,How can I handle social stress and anxiety,How can I handle social stress and anxiety
7,How can I stop putting off my assignments unti...,What can I do to prevent procrastination?,How can I manage my time more effectively?,What can I do to prevent procrastination?,What can I do to prevent procrastination?,What can I do to prevent procrastination?
8,how can I balance fun and school work,How can I maintain a work and life balance,How can I improve my concentration while study...,How can I manage my time more effectively?,How can I maintain a work and life balance,How can I maintain a work and life balance
9,How do I handle feeling isolated while studyin...,How can I deal with homesickness at university?,How can I deal with homesickness at university?,How can I deal with homesickness at university?,How can I deal with homesickness at university?,How can I deal with homesickness at university?


In [ ]:
# Accuracy of each model in retrival over 13 questions
for model in TEST_MODELS:
  print(f"RESULT OF {model}:", (result_dataset["original_question"] == result_dataset[model]).mean().round(3))

RESULT OF bert-base-nli-mean-tokens: 0.615
RESULT OF all-distilroberta-v1: 0.846
RESULT OF sentence-t5-base: 0.923
RESULT OF all-MiniLM-L6-v2: 0.923


### 2.3 Best Model Usage

In [ ]:
# Create a local Vector Database
qdrant = QdrantClient(":memory:")

# Load the model
model = "all-MiniLM-L6-v2"
encoder = SentenceTransformer(model)

# Create a collection for the model with its embeddings
qdrant.recreate_collection(
    collection_name = model,
    vectors_config = models.VectorParams(
        size = encoder.get_sentence_embedding_dimension(),
        distance = models.Distance.COSINE
    )
)

# Add the data to model
for index, row in bert_dataset.iterrows():
  key = row['question']
  value = row['answer']
  id = index + 1

  add_value(model, key, value, id)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Enter a question
question = "Im feeling sad"

result = search(model, question)
result = result[0].payload['text']

print(f"QUESTION: {question}\nANSWER: {result}")

QUESTION: Im feeling sad
ANSWER: Combating loneliness involves reaching out and making connections through clubs, organizations, or social events tailored to your interests. Don't hesitate to seek support from counseling services, which can offer strategies to build social skills and connect with others.


## 3.- Deployment Enviroment with Streamlit

The following code creates a .py docuemnt within the Colab enviroment

In [3]:
 %%writefile app.py
import streamlit as st

from sentence_transformers import SentenceTransformer

from qdrant_client import models, QdrantClient

import pandas as pd
from datasets import load_dataset

#************************************************************* LOAD DATA
data = load_dataset("ManuelAlv/academic_conuseling")

# Main dataset
bert_dataset = data['dataset'].to_pandas()

# Dataset used to test the chatbot
test_dataset = data['test'].to_pandas()
test_dataset.columns = ["test_question", "original_question"]

#************************************************************* Create Functions
# function to add values
def add_value(collection, key, value, id):
  encoder = SentenceTransformer(collection)

  qdrant.upsert(
      collection_name = collection,
      wait=True,
      points = [
          models.PointStruct(
              id = id,
              vector = encoder.encode(key).tolist(),
              payload = {
                  'text': value,
                  'question': key
              }
          )
      ]
  )

# Function to search for a value
def search(collection, query):
  search = qdrant.search(
      collection_name = collection,
      query_vector = encoder.encode(query).tolist(),
      limit = 1
  )
  return search

#************************************************************* Create VD
# Create a local Vector Database
qdrant = QdrantClient(":memory:")

# Load the model
model = "all-MiniLM-L6-v2"
encoder = SentenceTransformer(model)

# Create a collection for the model with its embeddings
qdrant.recreate_collection(
    collection_name = model,
    vectors_config = models.VectorParams(
        size = encoder.get_sentence_embedding_dimension(),
        distance = models.Distance.COSINE
    )
)

# Add the data to model
for index, row in bert_dataset.iterrows():
  key = row['question']
  value = row['answer']
  id = index + 1

  add_value(model, key, value, id)

# ************************************************************* QUERY

def get_response(input):
    result = search(model, input)
    result = result[0].payload['text']
    return result

st.set_page_config(page_title="UniSA Academic Support", page_icon="🧊", layout="wide")
st.title("UniSA Academic Support")

# with st.sidebar:
#     st.header("Settings")
#     st.text_input("Enter a website URL")

if 'conversation_ended' not in st.session_state:
    st.session_state['conversation_ended'] = False

if not st.session_state['conversation_ended']:
  with st.chat_message("AI"):
    st.write("Hi! I'm BrainHug AI, your supportive AI friend.")
    st.write("Feel free to chat with me at any time, just enter your question. If I can't answer, try rephrasing it again.")
    st.write("To finish conversation say: 'BYE'")

  user_q = st.chat_input("Start typing here")

  if user_q:
    if user_q.upper() == "BYE":
      st.session_state['conversation_ended'] = True
      with st.chat_message("AI"):
        st.write("Goodbye! Feel free to come back anytime.")
      st.stop()

    elif user_q is not None or user_q is "":
      response = get_response(user_q)
      with st.chat_message("Human"):
          st.write(user_q)
      with st.chat_message("AI"):
          st.write(response)
else:
    st.write("The conversation has ended. Refresh the page to start over.")

# with st.chat_message("AI"):
#     st.write("Hi!")
#     st.write("I'm BrainHug AI, your supportive AI friend.")
#     st.write("Feel free to chat with me at any time, just enter your question. If I can't answer, try rephrasing it again.")
#     st.write("If you want to finish the conversation, just say BYE")

# user_q = st.chat_input("Start typing here")
# if user_q:
#     if user_q.upper() == "BYE":  # Check if the user says "BYE" (case-insensitive)
#         with st.container():
#             st.markdown("Goodbye! Feel free to come back anytime.")
#         st.stop()

# elif user_q is not None or user_q is "":
#     response = get_response(user_q)
#     with st.chat_message("Human"):
#         st.write(user_q)
#     with st.chat_message("AI"):
#         st.write(response)

Writing app.py


The following line of code is going to get your IP address  
This is required to be able to access the local page


In [4]:
# This is going to be your code to access portal
print("THIS IS YOUR CODE:")
!wget -q -O - ipv4.icanhazip.com

THIS IS YOUR CODE:
34.106.124.68


Now just click on the web page follwed after:  
"your url is:"

In [ ]:
# You need to acess URL by clicking on link and enter the previous code
# ******  "your url is:"  ******
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.124.68:8501

npx: installed 22 in 4s
your url is: https://rotten-jeans-pump.loca.lt
/content/app.py:106: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif user_q is not None or user_q is "":
Generating dataset split: 100% 25/25 [00:00<00:00, 840.71 examples/s]
Generating test split: 100% 13/13 [00:00<00:00, 6643.03 examples/s]
modules.json: 100% 349/349 [00:00<00:00, 1.30MB/s]
config_sentence_transformers.json: 100% 116/116 [00:00<00:00, 476kB/s]
README.md: 100% 10.7k/10.7k [00:00<00:00, 33.9MB/s]
sentence_bert_config.json: 100% 53.0/53.0 [00:00<00:00, 234kB/s]
config.json: 100% 612/612 [00:00<00:00, 2.73MB/s]
pytorch_model.bin: 100% 90.9M/90.9M [00:00<00:00, 172MB/s]
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage clas